In [1]:
import tensorflow as tf

<h3 style='color:purple'>Create tf dataset from a list</h3>

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

<h3 style='color:purple'>Iterate through tf dataset</h3>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


<h3 style='color:purple'>Iterate through elements as numpy elements</h3>

In [4]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


<h3 style='color:purple'>Iterate through first n elements in tf dataset</h3>

In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


<h3 style='color:purple'>Filter sales numbers that are < 0</h3>

In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


<h3 style='color:purple'>Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->72 conversation rate</h3>

In [7]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


<h3 style='color:purple'>Shuffe</h3>

In [8]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2232
2448


https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset

<h3 style='color:purple'>Batching</h3>

In [9]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1584 1512]
[2232 2448]
[2304 2232]


<h3 style='color:purple'>Perform all of the above operations in one shot</h3>

In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 2232]
[1584 2304]
[2448 2232]


<h3>Images</h3>

In [11]:
images_ds = tf.data.Dataset.list_files('./dataset/images/*/*', shuffle=False)

In [12]:
image_count = len(images_ds)
image_count

130

In [13]:
type(images_ds)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [14]:
for file in images_ds.take(3):
    print(file.numpy())

b'.\\dataset\\images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'.\\dataset\\images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'.\\dataset\\images\\cat\\A cat appears to have caught the....jpg'


In [15]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'.\\dataset\\images\\dog\\Largest Dog Breeds \xe2\x80\x93 American Kennel Club.jpg'
b'.\\dataset\\images\\dog\\How to make your dog feel comfortable....jpg'
b'.\\dataset\\images\\dog\\The Black Dog Tavern Company _ Life off....jpg'


In [16]:
class_names = ["cat","dog"]

In [17]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [18]:
len(train_ds)

104

In [19]:
len(test_ds)

26

In [20]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [24]:
get_label("./dataset/images//dog\\20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'./dataset/images//dog'>

In [22]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [25]:
img, label = process_image("./dataset/images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

array([[[155.      , 186.      , 215.      ],
        [156.      , 187.      , 216.      ],
        [158.      , 189.      , 218.      ],
        [160.0039  , 189.0039  , 219.0039  ],
        [161.0039  , 190.0039  , 220.0039  ],
        [162.      , 191.      , 221.      ],
        [166.      , 193.      , 222.      ],
        [167.      , 194.      , 223.      ],
        [168.      , 195.      , 224.      ],
        [169.      , 196.      , 225.      ],
        [170.      , 197.      , 224.      ],
        [170.0039  , 197.0039  , 224.0039  ],
        [172.      , 199.      , 226.      ],
        [173.0039  , 199.0039  , 224.0039  ],
        [174.0039  , 200.0039  , 225.0039  ],
        [175.0039  , 201.0039  , 226.0039  ],
        [176.      , 202.      , 227.      ],
        [177.0039  , 203.0039  , 228.0039  ],
        [177.0039  , 203.0039  , 228.0039  ],
        [179.0039  , 203.0039  , 227.0039  ],
        [180.0039  , 204.0039  , 228.0039  ],
        [180.0039  , 204.0039  , 2

In [26]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [27]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[ 70.        59.        53.      ]
  [ 71.78052   61.780518  59.780518]
  [ 72.93628   64.93628   61.93628 ]
  ...
  [ 45.398438  38.960938  20.679688]
  [ 47.89917   44.89917   27.89917 ]
  [ 55.41382   53.41382   41.41382 ]]

 [[ 70.76636   59.766357  53.766357]
  [ 73.32031   63.320312  61.320312]
  [ 73.80884   65.80884   62.808838]
  ...
  [ 65.29736   59.679688  47.679688]
  [ 68.078125  64.078125  52.757812]
  [ 74.48511   71.48511   62.485107]]

 [[ 70.65625   59.65625   53.65625 ]
  [ 76.00415   66.00415   64.00415 ]
  [ 73.5625    65.5625    62.5625  ]
  ...
  [ 84.39844   80.39844   69.96094 ]
  [ 88.53125   85.53125   78.53125 ]
  [ 93.10034   90.10034   83.10034 ]]

 ...

 [[ 38.56372   35.56372   30.56372 ]
  [ 46.0271    41.0271    37.0271  ]
  [ 46.243896  37.318604  32.28125 ]
  ...
  [187.54858  173.58594  190.58594 ]
  [185.76562  171.89844  188.89844 ]
  [180.73438  170.       186.13281 ]]

 [[ 40.640625  35.640625  31.640625]
  [ 54.        46.   

In [28]:
def scale(image, label):
    return image/255, label

In [29]:
train_ds = train_ds.map(scale)

In [30]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.7538603  0.76954657 0.8166054 ]
****Label:  b'dog'
****Image:  [0.38673407 0.51222426 0.66311276]
****Label:  b'cat'
****Image:  [0.21533395 0.2074908  0.15651041]
****Label:  b'dog'
****Image:  [0.         0.10980392 0.35686275 1.        ]
****Label:  b'dog'
****Image:  [0.90539217 0.90539217 0.9132353 ]
****Label:  b'cat'
